# Lab 2: demo 📓

The purpose of this lab is to add the ability to decouple parsing/writing from creating the `RNA_Molecule` object. This is an encapsulatd step that will not change the way the user would interact with this utility, but rather make the interface more flexible and less susceptible to errors coming from changes in the future.

In [13]:
# -- necessary setup

import os,sys
sys.path.append(os.path.abspath('../src'))

from IO.RNA_IO import RNA_IO
from utils import pathify_pdb

## Reading from PDB File

In [14]:
rna_io=RNA_IO()
pdb_path=pathify_pdb('7EAF')
mol=rna_io.read(pdb_path, "PDB")

> note: Species with this name already exists, will return the same instance
RNA_Molecule 7EAF already exists in the species CALDANAEROBACTER SUBTERRANEUS SUBSP. TENGCONGENSIS; not added again


##  Writing the Output to a PDB-like format File

In [21]:
rna_io.write(mol, "demo.pdb",'PDB')

# with open("demo.pdb", "r") as f:
#     print(f.read())

RNA molecule written to demo.pdb


To check the file content, check the file [`demo.pdb`](./demo.pdb) in the same directory as this notebook (demo/)

##  Reading from the Written File

In [16]:
mol1=rna_io.read("demo.pdb", "PDB")

> note: Species with this name already exists, will return the same instance
RNA_Molecule 7EAF already exists in the species CALDANAEROBACTER SUBTERRANEUS SUBSP. TENGCONGENSIS; not added again


##  Writing again the Content to another File 
To verify the integrity of the reading and writing functions, the content will be written to a second file. By comparing the two files, we can ensure that the content is identical, confirming that the functions are correctly implemented and that the PDB format is followed accurately.

In [20]:
rna_io.write(mol1, "demo1.pdb",'PDB')

# with open("demo1.pdb", "r") as f:
#     print(f.read())

RNA molecule written to demo1.pdb


Instead of printing, checkout the file content in [`demo1.pdb`](./demo1.pdb) file found in the current directory.

## Coarse-Grained 3D Representation



In [18]:
cg_mol=rna_io.read(pdb_path, "PDB", coarse_grained=True, atom_name="C1'") #atom_name is optional, default is "C1'"
rna_io.write(cg_mol, "demo_cg.pdb",'PDB')

> note: Species with this name already exists, will return the same instance
RNA_Molecule 7EAF already exists in the species CALDANAEROBACTER SUBTERRANEUS SUBSP. TENGCONGENSIS; not added again
RNA molecule written to demo_cg.pdb


## Reading other PDB Entries

In [19]:
from utils import get_pdb_ids_from_fam
from Families.family import Family

pdb_ids=get_pdb_ids_from_fam('SAM')
molecules=[]
fam=Family(id='RF01510',name='2dG-I ')

for pdb_id in pdb_ids:
    rna_io=RNA_IO()
    rna=rna_io.read(pathify_pdb(pdb_id),format='PDB')
    molecules.append(rna)


Family with this id already exists, will link it to the existing family
> note: Species with this name already exists, will return the same instance
RNA_Molecule 5FK6 already exists in the species THERMOANAEROBACTER TENGCONGENSIS; not added again
> note: Species with this name already exists, will return the same instance
RNA_Molecule 5FK3 already exists in the species THERMOANAEROBACTER TENGCONGENSIS; not added again
> note: Species with this name already exists, will return the same instance
RNA_Molecule 5FKF already exists in the species THERMOANAEROBACTER TENGCONGENSIS; not added again
> note: Species with this name already exists, will return the same instance
RNA_Molecule 4KQY already exists in the species BACILLUS SUBTILIS; not added again
> note: Species with this name already exists, will return the same instance
RNA_Molecule 5FK4 already exists in the species THERMOANAEROBACTER TENGCONGENSIS; not added again
> note: Species with this name already exists, will return the same 

In [12]:
print(f'number of molecules in 2dG-I family: {len(molecules)}')
for x in molecules:
    print(f'type of item {x} in molecules: {type(x)}')

number of molecules in 2dG-I family: 29
type of item ID: 5FK6 Experiment: X-RAY DIFFRACTION THERMOANAEROBACTER TENGCONGENSIS species in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 3GX2 Experiment: X-RAY DIFFRACTION None in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 5FK3 Experiment: X-RAY DIFFRACTION THERMOANAEROBACTER TENGCONGENSIS species in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 5FKF Experiment: X-RAY DIFFRACTION THERMOANAEROBACTER TENGCONGENSIS species in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 4KQY Experiment: X-RAY DIFFRACTION BACILLUS SUBTILIS species in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 3GX6 Experiment: X-RAY DIFFRACTION None in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 5FK4 Experiment: X-RAY DIFFRACTION THERMOANAEROBACTER TENGCONGENSIS species in molecules: <class 'Structure.RNA_